## Preprocessing and Feature Extraction

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from textblob import TextBlob 

from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import nltk

from argparse import Namespace
from functools import partial
from pathlib import Path
from pprint import pprint

import os
import pickle
import random
import re
import string

In [ ]:
df = pd.read_csv("/Users/harveymiller/Documents/GitHub/text-success/Data/Ukraine Tweets.csv")

## Sentiment Analysis:

Steps taken:
- Converting emojis to text
- We decide to remove all the mentions and hashtagged words, as these will be analysed separately
- Remove Links, as these don't contribute to SA
- Conducting the SA on our preprocessed data

Creating a new column so that we can see the adjusted tweet and original versiom

In [ ]:
df.insert(loc=6,
          column='Adjusted Tweet',
          value=df['rendered_content'])

Converting emojis to text

In [ ]:
import emot
import emoji

def demote(text):
    text = emoji.demojize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(demote)

Removing mentions and hashtagged words

In [ ]:
import re

def remove_mentions_hashtags(text):
    text = re.sub("@[A-Za-z0-9_]+","", text)
    text = re.sub("#[A-Za-z0-9_]+","", text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_mentions_hashtags)

Removing links

In [ ]:
import re

def remove_links(text):
    text = re.sub('http://\S+|https://\S+', '', text)
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'bit.ly/\S+', '', text) # remove bitly links
    text = text.strip('[link]') # remove [links]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_links)

Insert Polarity Score Column

In [ ]:
df.insert(loc=7,
          column='Polarity Score',
          value=df['Adjusted Tweet'])

Sentiment Analysis using NLTK's VADER

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def sentiment_analysis(text):  
    text = sia.polarity_scores(text)
    return text

df['Polarity Score'] = df['Polarity Score'].apply(sentiment_analysis)
    

Creating columns for:
- Negative Score
- Neutral Score
- Positive Score
- Compound Score [-1,1]

In [ ]:
df.insert(loc=8,
          column='Negative Score',
          value=df['Polarity Score'])

df.insert(loc=9,
          column='Neutral Score',
          value=df['Polarity Score'])

df.insert(loc=10,
          column='Positive Score',
          value=df['Polarity Score'])

df.insert(loc=11,
          column='Compound Score',
          value=df['Polarity Score'])

In [ ]:
def negative_score(text):
    text = text['neg']
    return text

df['Negative Score'] = df['Negative Score'].apply(negative_score)

In [ ]:
def neutral_score(text):
    text = text['neu']
    return text

df['Neutral Score'] = df['Neutral Score'].apply(neutral_score)

In [ ]:
def positive_score(text):
    text = text['pos']
    return text

df['Positive Score'] = df['Positive Score'].apply(positive_score)

In [ ]:
def compound_score(text):
    text = text['compound']
    return text

df['Compound Score'] = df['Compound Score'].apply(compound_score)

In [ ]:
df.head()

Sentiment Analysis using TextBlob

In [ ]:
df.insert(loc=12,
          column='Polarity Score_textblob',
          value=df['Adjusted Tweet'])

In [ ]:
df.insert(loc=13,
          column='Subjectivity Score_textblob',
          value=df['Adjusted Tweet'])

In [ ]:
from textblob import TextBlob

#Create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

df['Polarity Score_textblob'] = df['Polarity Score_textblob'].apply(getPolarity)
df['Subjectivity Score_textblob'] = df['Subjectivity Score_textblob'].apply(getSubjectivity)


In [ ]:
df.head()

## Further manipulating the tweet

Steps taken:
- Lowercase
- Punctuation
- Tokenization
- Stopword filtering
- Stemming

Changing all text to lowercase

In [ ]:
def lowercase(text):    
    text = text.lower()
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(lowercase)

Removing all Punctuation

In [ ]:
import string

def punctuation_remove(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(punctuation_remove)

Tokenizing

In [ ]:
from nltk import word_tokenize

def tokenize(text):
    text = word_tokenize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(tokenize)

Stopword Filtering

In [ ]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stop_words]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_stopwords)

Stemming

In [ ]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def stem(text):
    stemmed = [porter.stem(word) for word in text]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(stem)

In [ ]:
df.head()

## Months since creation of account

In [ ]:
#Finding year of creation
df.insert(loc=5,
          column='Year of Creation',
          value=df['user_created'])

In [ ]:
from datetime import *

#return the year of account creation
def account_creation(text):
    text = datetime.strptime(text, "%Y-%m-%d %H:%M:%S+00:00")
    hour = str(text)[0:4]
    return int(hour)

df['Year of Creation'] = df['Year of Creation'].apply(account_creation)

In [ ]:
#Finding year of tweet
df.insert(loc=6,
          column='Year of Tweet',
          value=df['date'])

In [ ]:
from datetime import *

#return the year of tweet
df['Year of Tweet'] = df['Year of Tweet'].apply(account_creation)

## Time of Day

In [ ]:
#create a column for hours:
df.insert(loc=2,
          column='hour of tweet',
          value=df['date'])

In [ ]:
from datetime import *

#return the hour of the tweet
def hour(text):
    text = datetime.strptime(text, "%Y-%m-%d %H:%M:%S+00:00")
    hour = str(text.time())[0:2]
    return int(hour)

df['hour of tweet'] = df['hour of tweet'].apply(hour)

In [ ]:
#insert column for the time of day
df.insert(loc=3,
          column='time of day',
          value=df['hour of tweet'])

In [ ]:
def time_of_day(text):  
    if ((text < 4) & (text > 8 )):
        return 'Early Morning'
    elif ((text < 8) & (text > 12 )):
        return 'Morning'
    elif ((text < 12) & (text > 16 )):
        return 'Noon'
    elif ((text < 16) & (text > 20 )):
        return 'Eve'
    elif ((text < 20) & (text > 24 )):
        return 'Night'
    elif ((text < 24) & (text > 4 )):
        return 'Late Night'
    

df['time of day'] = df['time of day'].apply(time_of_day)


In [ ]:
df.tail()

## Video, GIF and Photo Count

Photo Count

In [ ]:
#Creating a photo count column...
df.insert(loc=23,
          column='Photo Count',
          value=df['media'])

In [ ]:
#Counting number of Photos in media column
#No need to tokenize
def photo_count(text):
    text = str(text)
    text = text.count('Photo')
    return text

df['Photo Count'] = df['Photo Count'].apply(photo_count)


Video Count

In [ ]:
#Creating a video count column...
df.insert(loc=24,
          column='Video Count',
          value=df['media'])

In [ ]:
#We need to tokenize the media column so that we can count how many videos there are...
from nltk import word_tokenize

def tokenize(text):
    text = str(text)
    text = word_tokenize(text)
    return text

df['Video Count'] = df['Video Count'].apply(tokenize)

In [ ]:
#Counting number of Videos in media column
def video_count(text):
    text = text.count('Video')
    return text

df['Video Count'] = df['Video Count'].apply(video_count)

Gif Count

In [ ]:
#Creating a GIF count column...
df.insert(loc=25,
          column='GIF Count',
          value=df['media'])

In [ ]:
#We need to tokenize the media column so that we can count how many GIFs there are...
from nltk import word_tokenize

def tokenize(text):
    text = str(text)
    text = word_tokenize(text)
    return text

df['GIF Count'] = df['GIF Count'].apply(tokenize)

In [ ]:
#Counting number of GIFs in media column
def gif_count(text):
    text = text.count('Gif')
    return text

df['GIF Count'] = df['GIF Count'].apply(gif_count)

## Creating our final Dataframe

Steps taken:
- Drop: media, date, user, polarity score

In [ ]:
df.drop(['media','date','user','Polarity Score'], axis=1)